# Current Infusion

In [1]:
import pyccl as ccl
import numpy as np
import fitsio
from fitsio import FITS, FITSHDR
import healpy as hp
import time
import matplotlib.pyplot as plt
import pyccl.ccllib as lib
import pyccl.nl_pt as pt
from astropy.io import fits
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
import os

In [27]:
path_in = "/global/cfs/cdirs/lsst/groups/CS/mass_sheets/"
directory_contents = os.listdir(path_in)
print("*** original mass sheets ***")
print(directory_contents)

zfile = np.loadtxt("z2ts.txt", delimiter=",")
snaplist = zfile[:, 1].astype(int)
zlist = zfile[:, 0]

# Choosing an arbitrary snapshot and reading its redshift
snap_number = 487
index = np.where(snaplist == snap_number)[0][0]
z_snap = zlist[index]
print(z_snap)

*** original mass sheets ***
['density_map_411_dens.bin', 'density_map_442_dens.bin', 'density_map_307_dens.bin', 'density_map_198_dens.bin', 'density_map_259_dens.bin', 'density_map_373_dens.bin', 'density_map_315_dens.bin', 'density_map_355_dens.bin', 'density_map_272_dens.bin', 'density_map_279_dens.bin', '.ipynb_checkpoints', 'density_map_464_dens.bin', 'density_map_247_dens.bin', 'density_map_286_dens.bin', 'density_map_241_dens.bin', 'density_map_189_dens.bin', 'density_map_219_dens.bin', 'density_map_224_dens.bin', 'density_map_203_dens.bin', 'density_map_121_dens.bin', 'density_map_176_dens.bin', 'density_map_230_dens.bin', 'density_map_475_dens.bin', 'density_map_331_dens.bin', 'density_map_137_dens.bin', 'density_map_194_dens.bin', 'density_map_347_dens.bin', 'density_map_323_dens.bin', 'density_map_213_dens.bin', 'density_map_180_dens.bin', 'density_map_167_dens.bin', 'density_map_392_dens.bin', 'density_map_151_dens.bin', 'density_map_184_dens.bin', 'density_map_141_dens.bi

In [4]:
cosmo_ccl = ccl.Cosmology(Omega_c=0.239, Omega_b=0.047, h=0.7, sigma8=0.82, n_s=0.96)

Om_m = cosmo_ccl["Omega_m"]
rho_crit = lib.cvar.constants.RHO_CRITICAL
Aia = 1.0

### Function's definitions

In [5]:
def Epsilon1_NLA(cosmo, z, A1, rho_crit, sxx, syy):
    gz = ccl.growth_factor(cosmo, 1.0 / (1 + z))
    Fact = -1 * A1 * 5e-14 * rho_crit * cosmo["Omega_m"] / gz
    # e1_NLA =  - Fact  * (sxx - syy)
    e1_NLA = Fact * (sxx - syy)
    return e1_NLA


def Epsilon2_NLA(cosmo, z, A1, rho_crit, sxy):
    gz = ccl.growth_factor(cosmo, 1.0 / (1 + z))
    Fact = -1 * A1 * 5e-14 * rho_crit * cosmo["Omega_m"] / gz
    e2_NLA = 2 * Fact * sxy
    # e2_NLA = Fact * sxy
    return e2_NLA


def compute_tidal_tensor_spherical_new(hpmap, smoothing=0, downsample_nside=2048):

    # -----------------------
    # Get alm from delta map:
    # The input maps must all be in ring ordering.

    lmax = 2048 * 4

    alm_E = hp.sphtfunc.map2alm(
        hpmap.real,
        lmax=2048 * 4,
        mmax=None,
        iter=3,
        pol=False,
        use_weights=False,
        datapath=None,
    )
    alm_B = alm_E * 0.0
    ##We set the B modes to 0
    ell, emm = hp.Alm.getlm(lmax=lmax)

    kalmsE = alm_E / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
    kalmsE[ell == 0] = 0.0

    kalmsB = alm_B / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
    kalmsB[ell == 0] = 0.0

    # if (smoothing > 0.0) :
    #     #smooth by sigma, given in radians
    #     alm_E_smooth=hp.sphtfunc.smoothalm(alm_E, fwhm=0.0, sigma=smoothing, beam_window=None, pol=False, mmax=None, verbose=True, inplace=True)
    #     alm_E = alm_E_smooth
    #     del alm_E_smooth

    nside = hp.get_nside(hpmap)
    # maps_QU = hp.alm2map_spin((alm_E, alm_B), nside, spin=2,lmax=2048*4,mmax=None)
    # maps_QU = hp.alm2map_spin((kalmsE, kalmsB), nside, spin=2,lmax=lmax,mmax=None)
    _, gamma1, gamma2 = hp.alm2map(
        [kalmsE * 0.0, kalmsE, kalmsB], nside=int(downsample_nside), pol=True
    )
    maps_QU = gamma1, gamma2
    hpmap = hp.ud_grade(map_in=hpmap, nside_out=int(downsample_nside))
    delta = np.copy(hpmap)
    # In 2D cartesian, s11(k)+s22(k)=delta(k)
    # In the spherical polarization framework, we have:
    # Q is 'g1' = (t11 - t22)
    # delta is t11 + t22
    # U is 'g2' = t12
    # t11 is (Q +delta)/2
    # t22 is (delta-Q)/2
    # Need to subtract off the trace, affecting s_11 and s_22...
    tidal_tensor_sph = np.zeros((hp.nside2npix(downsample_nside), 3), dtype=np.float32)

    # t11
    tidal_tensor_sph[:, 0] = (maps_QU[0] + delta) / 2.0 - 1.0 / 3 * hpmap

    # t22
    tidal_tensor_sph[:, 1] = (delta - maps_QU[0]) / 2.0 - 1.0 / 3 * hpmap

    # t12
    tidal_tensor_sph[:, 2] = maps_QU[1]

    Norm_empirical = (np.pi / 2) ** 2

    tidal_tensor_sph *= Norm_empirical

    return tidal_tensor_sph

### Initialization

In [6]:
fn_density_map = "density_map_411_dens.bin"
print(f"Processing mass sheet {fn_density_map}...")
density_map_filename = f"{path_in}{fn_density_map}"
# read mass density map from the mass sheet file
tmp = np.fromfile(density_map_filename, "<f")
density_map = hp.pixelfunc.ud_grade(
    tmp, 4096, pess=False, order_in="NESTED", order_out="RING", power=None, dtype=None
)

density_map /= np.mean(density_map)  # divide by average density
density_map -= 1  # compute overdensity

Processing mass sheet density_map_411_dens.bin...


### Computing IA maps starting from the density map

In [7]:
s_joachim = compute_tidal_tensor_spherical_new(density_map)

/tmp/ipykernel_2019692/1331318071.py:38: RuntimeWarning: divide by zero encountered in divide
  kalmsE = alm_E / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
/tmp/ipykernel_2019692/1331318071.py:38: RuntimeWarning: invalid value encountered in divide
  kalmsE = alm_E / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
/tmp/ipykernel_2019692/1331318071.py:41: RuntimeWarning: divide by zero encountered in divide
  kalmsB = alm_B / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
/tmp/ipykernel_2019692/1331318071.py:41: RuntimeWarning: invalid value encountered in divide
  kalmsB = alm_B / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)


In [8]:
s_11_joachim = s_joachim[:, 0]
s_22_joachim = s_joachim[:, 1]
s_12_joachim = s_joachim[:, 2]

In [ ]:
IA_NLA = Epsilon1_NLA(
    cosmo=cosmo_ccl,
    z=z_snap,
    rho_crit=rho_crit,
    A1=1.0,
    sxx=s_11_joachim,
    syy=s_22_joachim,
), Epsilon2_NLA(cosmo=cosmo_ccl, z=z_snap, A1=1.0, rho_crit=rho_crit, sxy=s_12_joachim)

# Alternative Infusion

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.cosmology import z_at_value
from bornraytrace import intrinsic_alignments as ia
from bornraytrace import lensing
from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")
rcParams.update({"font.size": 16})
rcParams.update({"image.cmap": "inferno"})

### Functions definitions

In [18]:
def F_nla(z, om0, A_ia, rho_crit, eta=0.0, z0=0.0, lbar=0.0, l0=1e-9, beta=0.0):
    prefactor = -A_ia * 5e-14 * rho_crit * om0
    inverse_linear_growth = 1.0 / ccl.growth_factor(cosmo_ccl, 1.0 / (1 + z))
    redshift_dependence = ((1 + z) / (1 + z0)) ** eta
    luminosity_dependence = (lbar / l0) ** beta
    Norm_empirical = (np.pi / 2) ** 2

    return (
        prefactor
        * inverse_linear_growth
        * redshift_dependence
        * luminosity_dependence
        * Norm_empirical
    )


def kappa2shear(kappa_map, lmax, downsample_nside=None):
    """
    Performs inverse Kaiser-Squires on the sphere with healpy spherical harmonics

    :param kappa_map: healpix format complex convergence (kappa) map
    :param lmax: maximum multipole
    :param downsample: option to downsample map output. A good compromise choice to reduce nside by half is: lmax=nside*2, downsample_nside=nside/2
    :return: complex shear map (gamma1 + 1j * gamma2)
    """

    nside = hp.npix2nside(len(kappa_map))

    almsE = hp.map2alm(kappa_map.real, lmax=lmax, pol=False)
    almsB = hp.map2alm(kappa_map.imag, lmax=lmax, pol=False)
    ell, emm = hp.Alm.getlm(lmax=lmax)

    kalmsE = almsE / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
    kalmsE[ell == 0] = 0.0

    kalmsB = almsB / (1.0 * ((ell * (ell + 1.0)) / ((ell + 2.0) * (ell - 1))) ** 0.5)
    kalmsB[ell == 0] = 0.0

    if downsample_nside is None:
        _, gamma1, gamma2 = hp.alm2map(
            [kalmsE * 0.0, kalmsE, kalmsB], nside=int(nside), pol=True
        )
    else:
        assert downsample_nside <= nside
        _, gamma1, gamma2 = hp.alm2map(
            [kalmsE * 0.0, kalmsE, kalmsB], nside=int(downsample_nside), pol=True
        )

    return gamma1 + 1j * gamma2

In [21]:
# Compute NLA prefactor
f_nla_array = F_nla(z_snap, Om_m, A_ia=1, rho_crit=rho_crit)
# Apply NLA prefactor to the density map to obtain the kappa_IA map
kappa_ia_array = hp.ud_grade(density_map, hp.npix2nside(len(density_map))) * f_nla_array
# Apply inverse Kaiser-Squires to obtain the IA map
ia_alt = lensing.kappa2shear(kappa_ia_array, lmax=2048 * 4, downsample_nside=2048)

## Comparison

In [28]:
IA_NLA[0] / ia_alt.real

array([1.00000002, 1.00000006, 1.00000011, ..., 1.00000003, 1.00000002,
       1.00000009])

In [29]:
IA_NLA[1] / ia_alt.imag

array([2.00000001, 2.00000004, 1.99999998, ..., 1.99999986, 1.99999986,
       1.99999988])